In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', 500)

In [2]:
data = pd.read_csv('energydata_complete.csv')
data.head(2)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.7900,19.79,44.73,19.0,45.566667,17.166667,55.2,7.026667,84.256667,17.2,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.7225,19.79,44.79,19.0,45.992500,17.166667,55.2,6.833333,84.063333,17.2,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195


In [3]:
data.drop(['date', 'lights'], axis=1, inplace=True)

In [4]:
df = data.iloc[:, :-15]
X = df.drop(columns=['Appliances', 'T1', 'RH_1', 'RH_2', 'T3', 'RH_3', 'RH_4', 'T5', 'RH_5', 'T6', 'T4'])
y = df['T6']

In [5]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
reg = LinearRegression()
model = reg.fit(X_train, y_train)

# Predictions
pred = reg.predict(X_test)

In [7]:
r2 = r2_score(y_test, pred)
round(r2, 3)

0.643

In [8]:
# Scale the data
scaler = MinMaxScaler()
norm_df = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

# Create the X and y variables
features = norm_df.drop(columns=['Appliances'])
target = norm_df['Appliances']

# Split the dataset
x_train, x_test, Y_train, Y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [9]:
reg_1 = LinearRegression()
model_1 = reg_1.fit(x_train, Y_train)

# Predictions
pred_1 = reg_1.predict(x_test)

In [10]:
# Calculate mae
mae = mean_absolute_error(Y_test, pred_1)
round(mae, 2)

0.05

In [11]:
# Calculate Residual sum of squares
rss = np.sum(np.square(Y_test - pred_1))
round(rss, 2)

45.35

In [12]:
# Calculate rmse
rmse = np.sqrt(mean_squared_error(Y_test, pred_1))
round(rmse, 3)

0.088

In [13]:
# Compare Regulation effects
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [14]:
linear_model_weights = get_weights_df(model_1, x_train, 'Linear_Model_Weight')
linear_model_weights

,Features,Linear_Model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [15]:
Rid = Ridge(alpha=0.4)
Rid.fit(x_train, Y_train)
Rid_pred = Rid.predict(x_test)

In [16]:
rmse = np.sqrt(mean_squared_error(Y_test, Rid_pred))
round(rmse, 3)

0.088

In [17]:
Las = Lasso(alpha=0.001)
Las.fit(x_train, Y_train)
Las_pred = Las.predict(x_test)

In [18]:
Lasso_weight_df = get_weights_df(Las, x_train, 'Lasso_Weight')
Lasso_weight_df

,Features,Lasso_Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [19]:
not_zero = Lasso_weight_df[Lasso_weight_df['Lasso_Weight'] != 0]
not_zero

,Features,Lasso_Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
24,Windspeed,0.002912
25,RH_1,0.017880


In [20]:
rmse = np.sqrt(mean_squared_error(Y_test, Las_pred))
round(rmse, 3)

0.094